#  [Visualizing India's SARS-CoV2 vaccination progress](https://www.moad.computer/blog/visualizing-indias-covid19-vaccination-progress)
## Author: [Dr. Rahul Remanan](https://www.linkedin.com/in/rahulremanan/)
## CEO, [Moad Computer](https://moad.computer)

[**Run this in Google Colab**](https://colab.research.google.com/github/MoadComputer/covid19-visualization/blob/main/examples/Visualizing_India_COVID19_vaccination_progress.ipynb)

This interactive dashboard to visualize the India's progress in SARS-CoV2 vaccination status is built using [Altair](https://altair-viz.github.io/), a declarative visualization library in Python.

The state-wise and union-territory-wise SARS-CoV2 vaccination statistics for India is sourced from the [Indian government's ministry of health and family welfare website](https://mohfw.gov.in).

**(Update on 5th July, 2024: This interactive dashboard will no longer be updated due to a lack of publicly available SARS-CoV2 vacination statistics from the [Indian government's ministry of health and family welfare website](https://mohfw.gov.in) that was used as the data source for this application.)**

This notebook is divided into two parts:


*   Part 01 -- Demo geospatial data visualization using Altair 
*   Part 02 -- Visualizing vaccination statistics of India



In [ ]:
DATA_UPDATE_DATE = '19-June-2024'
DATA_SOURCE = 'https://mohfw.gov.in [No new data from July 5th, 2024 onwards]'

# Setup

## Helper functions

In [ ]:
#@title
def colab_mode():
  try:
    from google import colab
    return True
  except:
    return False

def kaggle_mode():
  try:
    import kaggle_datasets
    return True
  except:
    return False

def apply_corrections(input_df):
  for state in list(input_df['state'].values):
    input_df.loc[input_df['state']==state,'state']=re.sub('[^A-Za-z ]+', '',str(state))
  input_df.loc[input_df['state']=='Karanataka','state']='Karnataka'
  input_df.loc[input_df['state']=='Himanchal Pradesh','state']='Himachal Pradesh'
  input_df.loc[input_df['state']=='Telengana','state']='Telangana'
  input_df.loc[input_df['state']=='Dadra and Nagar Haveli','state']='Dadra and Nagar Haveli and Daman and Diu'
  input_df.loc[input_df['state']=='Dadar Nagar Haveli','state']='Dadra and Nagar Haveli and Daman and Diu'
  input_df.loc[input_df['state']=='Dadra Nagar Haveli','state']='Dadra and Nagar Haveli and Daman and Diu'
  input_df.loc[input_df['state']=='Daman & Diu','state']='Dadra and Nagar Haveli and Daman and Diu'
  input_df.loc[input_df['state']=='Daman and Diu','state']='Dadra and Nagar Haveli and Daman and Diu'
  return input_df

def json_writer(json_input, json_output='output.json'):
  with open(json_output, 'w') as f:
    f.write(json_input)

def custom_tooltips():
  return [{'field' :'properties.state',
           'type'  :'nominal',
           'title' : 'State'},
          {'field' :'properties.fully_vaccinated_percentage',
           'type'  :'quantitative',
           'title' :'Fully vaccinated (%)'},
          {'field' :'properties.partly_vaccinated_percentage',
           'type'  :'quantitative',
           'title' :'Single dose (%)'},
          {'field' :'properties.dose_1',
           'type'  :'quantitative',
           'title' :'1st dose administered'},
          {'field' :'properties.dose_2',
           'type'  :'quantitative',
           'title' :'2nd dose administered'},
          {'field' :'properties.population',
           'type'  :'quantitative',
           'title' :'Total population'},
          {'field' :'properties.update_date',
           'type'  :'nominal',
           'title' :'Updated on'},
          {'field' :'properties.data_source',
           'type'  :'nominal',
           'title' :'Data from'}]

In [ ]:
if colab_mode():
  !python3 -m pip uninstall pyarrow -y
  !python3 -m pip install pyarrow
  !python3 -m pip install -q git+https://github.com/altair-viz/altair
  !python3 -m pip install -q geopandas
if kaggle_mode():
  !python3 -m pip install -q vega_datasets

# Import libraries

In [ ]:
import re, json, geopandas, altair as alt, numpy as np, pandas as pd
from tqdm import tqdm
from vega_datasets import data

# Part 01 -- [Demo visualization of US airport locations](https://github.com/altair-viz/altair_notebooks/blob/master/notebooks/examples/airports.ipynb)
This is an example geospatial data visualization using Altair. 

It demonstrates Altair's ability to interactively superimpose relevant statistics on corresponding geographical data.

The code below is adapted from the [official example notebook in Altair library](https://github.com/altair-viz/altair_notebooks/).

To minimize data dependencies of this example script, Altair's built-in airport location data and mapping information are used here.

In [ ]:
airports = data.airports.url
states = alt.topo_feature(data.us_10m.url, feature='states')

# The state-wise map of the US as background
background = alt.Chart(states).mark_geoshape(
    fill='lightblue',
    stroke='white'
).properties(
    width=500,
    height=300
).project('albersUsa')

# Overlay airport counts on the background
points = alt.Chart(airports).transform_aggregate(
    latitude='mean(latitude)',
    longitude='mean(longitude)',
    count='count()',
    groupby=['state']
).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.Size('count:Q', title='Number of Airports'),
    color=alt.value('red'),
    tooltip=['state:N', 'count:Q']
).properties(
    title='Number of airports in each US state'
)

background + points

# Part 02 -- Visualizing SARS-CoV2 vaccination progress in India

This is an interactive dashboard for visualizing the India's progress in SARS-CoV2 vaccinations.

## Read map data and vaccination statistics
The latest SARS-CoV2 vaccination data for India is obtained from the [Indian government's ministry of health and family welfare (MoHFW) website](https://mohfw.gov.in). To simplify the data-pipeline and make future usage more convenient, a portion of this data is also cached in the project's [GitHub repo](https://github.com/MoadComputer/covid19-visualization).

In [ ]:
REPO_URL = 'https://github.com/MoadComputer/covid19-visualization/raw/main/data'

India_statewise = geopandas.read_file(
  f'{REPO_URL}/GeoJSON_assets/India.geojson')
India_stats = pd.read_csv(
  f'{REPO_URL}/Coronavirus_stats/India/Population_stats_India_statewise.csv')
covid19_data = pd.read_csv(
  f'{REPO_URL}/Coronavirus_stats/India/COVID19_India_statewise.csv')
preds_df = pd.read_csv(
  f'{REPO_URL}/Coronavirus_stats/India/experimental/output_preds.csv')
India_vaccinations = pd.read_csv(
  f'{REPO_URL}/Coronavirus_stats/India/COVID19_vaccinations_India_statewise.csv')

India_statewise = apply_corrections(India_statewise)
India_stats = apply_corrections(India_stats)
India_vaccinations = India_vaccinations[India_vaccinations.state != 'Miscellaneous']
India_vaccinations = apply_corrections(India_vaccinations)

covid19_data = apply_corrections(covid19_data)

In [ ]:
json_writer(India_statewise.to_json(), 'India.geojson')

## Pre-process

In [ ]:
covid19_data = pd.merge(covid19_data, India_stats, on='state', how='left')
covid19_data = pd.merge(covid19_data, India_vaccinations, on='state', how='left')
covid19_data = pd.merge(India_statewise, covid19_data, on='state', how='left')

## Automated error correction
A very simple, yet effective error correction code to rectify some simple mistakes in the MoHFW sourced SARS-CoV2 vaccination statistics for India.

In [ ]:
def vac_dose_ecc(input_df):
  for s in tqdm(input_df['state']):
    dose_1 = int(input_df.loc[input_df['state']==s]['dose_1'].to_numpy()[0])
    dose_2 = int(input_df.loc[input_df['state']==s]['dose_2'].to_numpy()[0])
    if dose_1 < dose_2:
      print(
       f'\nFound an entry for: {s} with, \
       \n\t second doses ({dose_2}) greater than first doses ({dose_1}) ...')
      print('Whoops!!! Seems like a mathematical impossibility ...')
      print(f'Autoswitching dose 1 and dose 2 columns for: {s} ...')
      input_df.loc[input_df['state']==s, 'dose_1'] = dose_2
      input_df.loc[input_df['state']==s, 'dose_2'] = dose_1
  return input_df

covid19_data = vac_dose_ecc(covid19_data)

In [ ]:
covid19_data['fully_vaccinated_percentage'] = (
    (covid19_data['dose_2']/covid19_data['population'])*100).astype(np.uint8)
covid19_data['partly_vaccinated_percentage'] = (
    (covid19_data['dose_1']/covid19_data['population'])*100).astype(np.uint8)

covid19_data = covid19_data.fillna(0)

covid19_data['update_date'] = DATA_UPDATE_DATE
covid19_data['data_source'] = DATA_SOURCE

In [ ]:
covid19_data.head(2)

## Enable background transparency

In [ ]:
@alt.theme.register('transparent_theme', enable=True)
def transparent_theme(font=None):
    defaultFont="'Courier New', Courier, monospace"
    if font is not None:
        defaultFont=font
    return alt.theme.ThemeConfig({
    'config': {
      'background' : None,
      'style': {
#     'guide-label': {
#       'font': defaultFont,
#       'fontSize': 14.5,
#       'fontWeight': 'bold',
#     },
    'guide-title': {
      'font': defaultFont,
      'fontSize': 13,
      'fontWeight': 'bold',
    },
#     'group-title': {
#       'font': defaultFont,
#       'fontSize': 14.5,
#       'fontWeight': 'bold',
#     },
#   },
#   'title': {
#     'font': defaultFont,
#     'fontSize': 16.5,
#     'fontWeight': 'bold',
#     'dy': -3,
#     'anchor': 'start',

      },
    }
  }
                                )
alt.theme.enable('transparent_theme')

## Set Altair rendering options

In [ ]:
alt.renderers.enable(embed_options={'actions': False,
                                    'theme': 'transparent_theme',
                                    'fillOpacity': {'condition': {'param': 'select',
                                                                  'value': 1},
                                                    'value'    : 0.3},
                                     'tooltip'   : {'theme'    : 'dark',
                                                    'trigger'  :'mouseclick',
                                                    'highlight': {'select': {'type': 'mouseclick',
                                                                             'on'  : 'mouseclick'}}
                                                   }
                                   })

## Plot vaccine statistics on the map of India

In [ ]:
map_data = alt.Data(
    values=covid19_data.to_json(),
    format=alt.DataFormat(property='features', type='json')
)

In [ ]:
vac_plot_params_1 = alt.param(name='highlight', 
                              select={'type': 'point',
                                      'on'  : 'pointerover'})
vac_plot_params_2 = alt.param(name='select', select='point')

In [ ]:
vac_plot = alt.Chart(map_data).add_params(vac_plot_params_1,
                                          vac_plot_params_2
).mark_geoshape(blend='darken').encode(
    color={'field' : 'properties.fully_vaccinated_percentage',
           'type'  : 'quantitative',
           'title' : 'Fully vaccinated (%)',
           'scale' : alt.Scale(scheme='greens')},
    fillOpacity={'value': 1.0},
    opacity={
      'condition': [
        {
          'param': 'select',
          'empty': False,
          'value': 0.3
        },
        {
          'param': 'highlight',
          'empty': False,
          'value': 0.9
        }
      ],
      'value': 1.0
    },
    tooltip=custom_tooltips(),
).properties(width=480, height=560
).project('mercator')

In [ ]:
display(vac_plot)

## Save plot as HTML
The saved HTML plot can be used for web deployment. The [Vega embedding options](https://github.com/vega/vega-embed#Options) can be used to further customize the plots.

In [ ]:
vac_plot.save('India_vaccination.html',
              embed_options={'actions': False,
                             'theme' : 'transparent_theme',
                             'fillOpacity': {'condition': {'param': 'select',
                                                           'value': 1},
                                             'value'    : 0.3},
                             'tooltip'   : {'theme'    : 'dark',
                                            'trigger'  :'mouseclick',
                                            'highlight': {'select': {'type': 'mouseclick',
                                                                     'on'  : 'mouseclick'}}
                                            }
                             })